# Demonstrating statsmodels Modin Interoperability
###  Currently statsmodels is not completely interoperable with Modin. All the examples in this section are taken/ adapted from https://www.statsmodels.org/devel/gettingstarted.html or https://www.statsmodels.org/stable/index.html

In [1]:
import statsmodels.api as sm
import pandas
import modin.pandas as pd
from patsy import dmatrices

/Users/labanyamukhopadhyay/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/labanyamukhopadhyay/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


#### Example with sm.OLS()

In [2]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data
modin_df = pd.DataFrame(df)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2023-04-06 11:48:00,894	INFO worker.py:1553 -- Started a local Ray instance.


In [3]:
vars = ['Department', 'Lottery', 'Literacy', 'Wealth', 'Region']

modin_df = modin_df[vars]

modin_df[-5:]

Department  Lottery  Literacy  Wealth Region
81        Vienne       40        25      68      W
82  Haute-Vienne       55        13      67      C
83        Vosges       14        62      82      E
84         Yonne       51        47      30      C
85         Corse       83        49      37    NaN

In [4]:
modin_df = modin_df.dropna()

modin_df[-5:]

Department  Lottery  Literacy  Wealth Region
80        Vendee       68        28      56      W
81        Vienne       40        25      68      W
82  Haute-Vienne       55        13      67      C
83        Vosges       14        62      82      E
84         Yonne       51        47      30      C

In [5]:
y, X = dmatrices('Lottery ~ Literacy + Wealth + Region', data=modin_df, return_type='dataframe')

In [6]:
y = pd.DataFrame(y)
X = pd.DataFrame(X)

In [7]:
mod = sm.OLS(y, X)    # Describe model

ValueError: unrecognized data structures: <class 'modin.pandas.dataframe.DataFrame'> / <class 'modin.pandas.dataframe.DataFrame'>

In [8]:
res = mod.fit()       # Fit model

print(res.summary())

NameError: name 'mod' is not defined

sm.ols() is not interoperable with Modin currently.

#### Example with sm.ols(formula=)

In [9]:
modin_df = pd.DataFrame({"A": [10,20,30,40,50], "B": [20, 30, 10, 40, 50], "C": [32, 234, 23, 23, 42523]})

In [10]:
import statsmodels.formula.api as sm
result = sm.ols(formula="A ~ B + C", data=modin_df).fit()
print(result.params)

Intercept    14.952480
B             0.401182
C             0.000352
dtype: float64


In [11]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     1.375
Date:                Thu, 06 Apr 2023   Prob (F-statistic):              0.421
Time:                        11:48:10   Log-Likelihood:                -18.178
No. Observations:                   5   AIC:                             42.36
Df Residuals:                       2   BIC:                             41.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.9525     17.764      0.842      0.4

## Replicating statsmodels workflow with pandas

In [13]:
import statsmodels.api as sm

df = sm.datasets.get_rdataset("Guerry", "HistData").data
pandas_df = pandas.DataFrame(df)

In [14]:
vars = ['Department', 'Lottery', 'Literacy', 'Wealth', 'Region']

pandas_df = pandas_df[vars]

In [15]:
pandas_df = pandas_df.dropna()

In [16]:
y, X = dmatrices('Lottery ~ Literacy + Wealth + Region', data=df, return_type='dataframe')

In [17]:
y = pandas.DataFrame(y)
X = pandas.DataFrame(X)

In [18]:
mod = sm.OLS(y, X)    # Describe model

In [19]:
res = mod.fit()       # Fit model

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     6.636
Date:                Thu, 06 Apr 2023   Prob (F-statistic):           1.07e-05
Time:                        11:48:36   Log-Likelihood:                -375.30
No. Observations:                  85   AIC:                             764.6
Df Residuals:                      78   BIC:                             781.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      38.6517      9.456      4.087      

#### Example with sm.ols(formula=)

In [20]:
pandas_df = pd.DataFrame({"A": [10,20,30,40,50], "B": [20, 30, 10, 40, 50], "C": [32, 234, 23, 23, 42523]})

In [21]:
import statsmodels.formula.api as sm
result = sm.ols(formula="A ~ B + C", data=pandas_df).fit()
print(result.params)

Intercept    14.952480
B             0.401182
C             0.000352
dtype: float64


In [22]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     1.375
Date:                Thu, 06 Apr 2023   Prob (F-statistic):              0.421
Time:                        11:48:58   Log-Likelihood:                -18.178
No. Observations:                   5   AIC:                             42.36
Df Residuals:                       2   BIC:                             41.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.9525     17.764      0.842      0.4